# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Import API key

from api_keys import g_key

In [15]:

csv_path = os.path.join("Resources", "cities_weather_data_final.csv")
weather_df = pd.read_csv(csv_path)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Code
0,castro,-24.7911,-50.0119,293.56,79.0,1.0,2.97,BR
1,fortuna,40.5982,-124.1573,284.82,93.0,90.0,5.66,US
2,mar del plata,-38.0023,-57.5575,293.71,100.0,75.0,1.54,AR
3,hermanus,-34.4187,19.2345,295.37,44.0,74.0,1.79,ZA
4,korla,41.7597,86.1469,278.10,62.0,36.0,2.32,CN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [83]:
#Use the Lat and Lng as locations and Humidity as the weight.

locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"]

100.0

In [84]:
#Plot Heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, point_radius = 10, max_intensity = max(humidity))

In [85]:
#Add the layer to the map

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
hotel_df = weather_df.loc[(weather_df["Max Temp"] > 150) & (weather_df["Max Temp"] <= 300)
                                    & (weather_df["Humidity"] >= 50) & (weather_df["Humidity"] < 70)
                                    & (weather_df["Cloudiness"] > 25) & (weather_df["Wind Speed"] < 10)]

hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Code
4,korla,41.7597,86.1469,278.10,62.0,36.0,2.32,CN
8,ballina,-28.8667,153.5667,296.15,68.0,75.0,5.66,AU
11,uvinza,-5.1036,30.3911,299.78,68.0,57.0,1.25,TZ
14,punta arenas,-53.1500,-70.9167,283.15,66.0,75.0,6.17,CL
42,haines junction,60.7522,-137.5108,236.15,67.0,61.0,0.15,CA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
#Set Hotel column into the df

hotel_df["Hotel Name"] = ""
hotel_df.head()

/Users/dmontufar/opt/anaconda3/envs/pythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Code,Hotel Name
4,korla,41.7597,86.1469,278.10,62.0,36.0,2.32,CN,
8,ballina,-28.8667,153.5667,296.15,68.0,75.0,5.66,AU,
11,uvinza,-5.1036,30.3911,299.78,68.0,57.0,1.25,TZ,
14,punta arenas,-53.1500,-70.9167,283.15,66.0,75.0,6.17,CL,
42,haines junction,60.7522,-137.5108,236.15,67.0,61.0,0.15,CA,


In [78]:
#Find hotels within 5000 meters of the cities

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius" : "5000",
    "type" : "lodging",
    "key" : g_key
}

#Use iterrows to iterate thorugh pandas dataframe

for index, row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # get location from df
    location = f"{lat},{lng}"
    
    #Add location to params dict
    
    params["location"] = location
    
    #Assemble URL and make API request
    
    response = requests.get(base_url, params = params).json()
    
    #print(json.dumps(response, indent = 4))
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print(f"Missing result for {city}, skipping...")
    

Missing result for uvinza, skipping...
Missing result for albany, skipping...
Missing result for tangzhai, skipping...


In [79]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Code,Hotel Name
4,korla,41.7597,86.1469,278.10,62.0,36.0,2.32,CN,Gulisitan Hotel
8,ballina,-28.8667,153.5667,296.15,68.0,75.0,5.66,AU,Ramada Hotel & Suites by Wyndham Ballina Byron
11,uvinza,-5.1036,30.3911,299.78,68.0,57.0,1.25,TZ,
14,punta arenas,-53.1500,-70.9167,283.15,66.0,75.0,6.17,CL,Hotel Dreams Del Estrecho
42,haines junction,60.7522,-137.5108,236.15,67.0,61.0,0.15,CA,Parkside Inn
48,albany,42.6001,-73.9662,267.59,61.0,98.0,1.72,US,
54,mount isa,-20.7333,139.5000,299.15,69.0,100.0,5.88,AU,ibis Styles Mt Isa Verona
60,muros,42.7762,-9.0603,293.15,57.0,91.0,5.81,ES,Apartment Av. Castelao
82,ushuaia,-54.8000,-68.3000,285.15,58.0,75.0,7.20,AR,Albatros Hotel
131,lata,40.1629,-8.3327,292.04,56.0,94.0,6.26,PT,Casa Maquia


In [87]:
#Save DF 

csv_path = os.path.join("Resources", "hotels_data.csv")
hotel_df.to_csv(csv_path)

In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [104]:
# Add marker layer ontop of heat map

coordinates = list(zip(hotel_df["Lat"], hotel_df["Lng"]))
hotel_name = list(hotel_df["Hotel Name"])
markers = gmaps.marker_layer(coordinates, hover_text = hotel_name)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))